In [ ]:
import datetime
import os
import re
from collections import Counter
from datetime import datetime, timedelta

import numpy as np
from keras.preprocessing import sequence
import sys
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm, trange
import umap
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
from sklearn.model_selection import StratifiedKFold, TimeSeriesSplit
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
from scipy.cluster.hierarchy import dendrogram, linkage
import sklearn

In [ ]:
activity_mapping = {
                              "Cook_Breakfast": "Cook",
                              "Cook_Lunch": "Cook",
                              "Cook_Dinner": "Cook",
                              "Eat_Breakfast": "Eat",
                              "Eat_Lunch": "Eat",
                              "Eat_Dinner": "Eat",
                              "Morning_Meds": "Take_Medicine",
                              "Evening_Meds": "Take_Medicine",
                              "Wash_Breakfast_Dishes": "Wash_Dishes",
                              "Wash_Lunch_Dishes": "Wash_Dishes",
                              "Wash_Dinner_Dishes": "Wash_Dishes",
                              "Work_At_Table": "Work",
                              "Watch_TV": "Relax",
                              "Read": "Work",
                              "Entertain_Guests": "Relax",
                              "Sleep_Out_Of_Bed": "Relax",
                              "Step_Out": "Leave_Home",
                     }
FTWs = [720, 540, 360, 180, 60, 30, 15, 5, 3, 2, 1, 0, 0]
# FTWs = [0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144][::-1]
ftw_window = 10

In [ ]:

ann_dataset = pd.read_csv('../hh_dataset/hh102/hh102.ann.txt', sep='\t')

raw_columns = ['Date & Time', 'Sensor ID', 'Room-level', 'Sensor location', 'Message', 'Sensor Type']
ann_columns = raw_columns + ['Activity']

ann_dataset.columns = ann_columns

ann_dataset['Activity'] = ann_dataset['Activity'].apply(lambda x: activity_mapping[x] if x in activity_mapping else x)

activity2id = {}
count = 0
for act in ann_dataset['Activity'].unique():
    if act != 'Other_Activity':
        activity2id[act] = count
        count += 1
activity2id['Other_Activity'] = count
activity2id

In [ ]:
ann_dataset['Date & Time'] = pd.to_datetime(ann_dataset['Date & Time'], format='%Y-%m-%d %H:%M:%S')
start_time, end_time = ann_dataset['Date & Time'].min(), ann_dataset['Date & Time'].max()
timeframed_dataset = ann_dataset.set_index(['Date & Time'])

In [ ]:
## TODO reduce the time delta and compute the result between different time window

delta = timedelta(minutes=2)

number_of_time_window = int(np.ceil((end_time - start_time) / delta))
activities = np.zeros((number_of_time_window, len(activity2id)-1))
# timeframed_dataset[start_time: start_time+delta]
for i in range(number_of_time_window):
    activity_within_range = timeframed_dataset[start_time+i*delta: start_time+(i+1)*delta]['Activity'].unique()
    for j in activity_within_range:
        if j == "Other_Activity":
            continue
        else:
            activities[i][activity2id[j]] = 1

In [ ]:
def get_time_window(offset):
    return timeframed_dataset[start_time+offset*delta: start_time+(offset+1)*delta]

In [1]:
sensors_list = list(filter(lambda x : x[0] != 'T' , timeframed_dataset['Sensor ID'].unique()))
sensors_list
sensor2id = {sensor: i for i , sensor in enumerate(sensors_list)}

def is_triggered_sensor(tup):
    timestamp, sensor, message = tup
    if message in ['ON', 'OPEN']:
        return timestamp, sensor2id[sensor]
    try:
        if int(message) >=50:
            return timestamp, sensor2id[sensor]
    except:
        pass
    return timestamp, -1

def compute_sensor_activation(start_time, end_time, weight):
    sensor_activation = np.zeros(len(sensors_list))
    sensors_detail = list(timeframed_dataset[start_time: end_time][['Sensor ID', 'Message']].itertuples(name=None))
    # print(sensors_detail)
    for j in sensors_detail:
        timestamp, sensor_triggered = is_triggered_sensor(j)
        if sensor_triggered > -1:
            if weight == 'flat':
                weight = 1
            elif weight == 'left':
                weight = (timestamp-start_time)/(end_time-start_time)
            elif weight == 'right':
                weight = (end_time-timestamp)/(end_time-start_time)
            sensor_activation[sensor_triggered] = weight
    return sensor_activation

ftw_testing = np.zeros((ftw_window, len(sensors_list)))
i = 10
testing = timeframed_dataset[start_time+i*delta: start_time+(i+1)*delta]
t_star = start_time+(i+1)*delta
for i in range(ftw_window):
    l4, l3, l2, l1 = FTWs[i:i+4]
    l4, l3, l2, l1 = timedelta(minutes=l4), timedelta(minutes=l3), timedelta(minutes=l2), timedelta(minutes=l1)

    left_slope = compute_sensor_activation(t_star-l2, t_star-l1, weight='left')
    flat_part = compute_sensor_activation(t_star-l3, t_star-l2, weight='flat')
    right_slope = compute_sensor_activation(t_star-l4, t_star-l3, weight='right')

    ftw_testing[i] = np.maximum(np.maximum(left_slope, flat_part), right_slope)
np.where(ftw_testing > 0)
ftw_testing.shape

# ftw features (Shape: (number_of_time_window, ftw_window_size, no_sensors))
features = np.zeros((number_of_time_window, ftw_window, len(sensors_list)))
for i in trange(number_of_time_window):
    t_star = start_time+(i+1)*delta
    for j in range(ftw_window):
        l4, l3, l2, l1 = FTWs[j:j+4]
        l4, l3, l2, l1 = timedelta(minutes=l4), timedelta(minutes=l3), timedelta(minutes=l2), timedelta(minutes=l1)

        left_slope = compute_sensor_activation(t_star-l2, t_star-l1, weight='left')
        flat_part = compute_sensor_activation(t_star-l3, t_star-l2, weight='flat')
        right_slope = compute_sensor_activation(t_star-l4, t_star-l3, weight='right')

        features[i][j] = np.maximum(np.maximum(left_slope, flat_part), right_slope)

NameError: name 'timeframed_dataset' is not defined

In [ ]:
print(features.shape, activities.shape)

np.save('./ftw_data/fib_ftw_2mins_features.npy', features)
np.save('./ftw_data/fib_ftw_2mins_activities.npy', activities)